## Install Required Libraries

In [0]:
# !pip install pypdf2
# !pip install langchain==0.3.11
# !pip install langchain-openai==0.2.12
# !pip install langchain-community==0.3.11

## Export OpenAI Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Please enter your Open AI API Key here: ')

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Initial Imports

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


import PyPDF2

## Load the PDF and Extract Contents

In [0]:
def load_pdf(file_path):
    """Load and parse a PDF file."""
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        # Extract text from all pages
        study_material = ""
        for page in reader.pages:
            study_material += page.extract_text()
    return study_material

## Create the LLM Object

In [0]:
def create_llm(model_name):
    return ChatOpenAI(model_name=model_name, temperature=0.0)


## Create Merge Chain

In [0]:
def create_merge_chain(chatgpt):
    merge_prompt = ChatPromptTemplate.from_template(
    """Create a report about {topic} with the following information:
    
      Summary:
      {summary}
      Quiz Questions:
      {response}

      Report should be in the following format:

      Topic: <name of the topic>
      
      Summary: <Summary of the topic>

      Quiz Questions: <Quiz Questions>
    """)

    merge_chain = (
        merge_prompt
            |
        chatgpt
            |
        StrOutputParser()
    )
    return merge_chain
    

## Create Summarize Chain

In [0]:
def create_summarize_chain(chatgpt):
    summarize_prompt =  """Based on the following study_material: {study_material} generate a summary for the given topic: {topic}. 
    Headline for the summary should be Summary only and summary should not be more than 10 lines.

    Example:

    -Summary-
    Prompt engineering is a crucial practice in natural language processing (NLP) within artificial intelligence, 
    where text prompts guide AI models in performing specific tasks. 
    """

    summarize_prompt_template = ChatPromptTemplate.from_template(summarize_prompt)
    summarize_chain = (summarize_prompt_template
                  |
              chatgpt
                  |
              StrOutputParser())
    return summarize_chain

## Create Quiz Generation Chain

In [0]:
def create_quiz_chain(chatgpt):
    quiz_prompt = """
    Act as a Study Assistant for Quiz Question Generation.
    For the study_material ,summary and topics delimited below by triple backticks,
    create quiz questions based on the summary and study material provided. The heading of the quiz questions should be Quiz Questions.
    
    Study Material:
    ```{study_material}```
    Summary:
    ```{summary}```
    Topic:
    ```{topic}```
    
    examples of output:
    
    Summary:

    Prompt engineering refines inputs to language models for better output control.

    In agent-based systems, it helps control agent behavior and task performance.

    Useful in robotics and conversational AI.
    
    Quiz Questions:
    What is the primary goal of prompt engineering in agent-based systems?

    a) To optimize agent memory

    b) To refine inputs for better output control

    c) To improve agent hardware

    d) To increase computational power

    Answer: b) To refine inputs for better output control

    In which domain is prompt engineering most commonly used for enhancing agent performance?

    a) Image recognition

    b) Conversational AI

    c) Data processing

    d) Video editing

    Answer: b) Conversational AI
    
    
    """
    quiz_prompt_template = ChatPromptTemplate.from_template(quiz_prompt)
    quiz_chain = (quiz_prompt_template
                    |
                chatgpt
                    |
                StrOutputParser())
    return quiz_chain

## Create Sequential Chain

In [0]:
def create_seq_chain(chatgpt):
    
    summarize_chain = create_summarize_chain(chatgpt)
    quiz_chain = create_quiz_chain(chatgpt)
  
    seq_chain = (
    RunnablePassthrough.assign(summary=summarize_chain)
      |
    RunnablePassthrough.assign(response=quiz_chain)
    )
    
    return seq_chain

## Generate the Final Response

In [0]:
def create_final_response(seq_chain,merge_chain,study_material,topic):
    final_chain = (
    seq_chain
    |
    merge_chain
    )
    response = final_chain.invoke({'study_material': study_material,'topic': topic})
    return response

In [0]:
## Run the Code

In [0]:
study_material = load_pdf("./prompt_engineering.pdf")
# topic = "Prompt Engineering for Agents"
topic = "Define Prompting Techniques"
# chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.0)
chatgpt = create_llm('gpt-4o-mini')
seq_chain = create_seq_chain(chatgpt)
merge_chain = create_merge_chain(chatgpt)
response = create_final_response(seq_chain,merge_chain,study_material,topic)

In [0]:
from IPython.display import Markdown, display

display(Markdown(response))

**Topic:** Define Prompting Techniques

**Summary:**  
Prompting techniques are advanced methods used to enhance the effectiveness of AI prompts. Key techniques include Zero-Shot Prompting, where the model performs tasks without prior specific training; Few-Shot Prompting, which provides examples to improve task performance; and Chain-of-Thought (CoT), allowing the model to break down complex reasoning into intermediate steps. These techniques help in achieving more accurate and coherent outputs by guiding the model's understanding and response generation. Effective prompting is essential for maximizing the capabilities of AI models in various applications.

**Quiz Questions:**

1. What is the purpose of prompt engineering in natural language processing (NLP)?
   - a) To create complex algorithms
   - b) To describe the task the AI should perform
   - c) To enhance hardware performance
   - d) To increase data storage capacity  
   **Answer:** b) To describe the task the AI should perform

2. Which of the following best defines a "prompt"?
   - a) A random question posed to the AI
   - b) A detailed description of the desired output from an AI model
   - c) A set of instructions for programming
   - d) A summary of AI capabilities  
   **Answer:** b) A detailed description of the desired output from an AI model

3. What is an example of a Few-Shot Prompting technique?
   - a) Asking the model to perform a task without any examples
   - b) Providing multiple examples to guide the model's response
   - c) Giving the model a single instruction
   - d) Allowing the model to generate responses freely  
   **Answer:** b) Providing multiple examples to guide the model's response

4. Which prompting technique allows the model to break down complex reasoning into intermediate steps?
   - a) Zero-Shot Prompting
   - b) Few-Shot Prompting
   - c) Chain-of-Thought (CoT)
   - d) Open-Ended Prompting  
   **Answer:** c) Chain-of-Thought (CoT)

5. What should be avoided when creating prompts for AI models?
   - a) Providing clear and concise instructions
   - b) Using open-ended questions
   - c) Specifying output formats
   - d) Maintaining a consistent tone  
   **Answer:** b) Using open-ended questions

6. How does Zero-Shot Prompting differ from Few-Shot Prompting?
   - a) Zero-Shot Prompting requires examples, while Few-Shot does not.
   - b) Zero-Shot Prompting does not rely on specific training, while Few-Shot provides examples.
   - c) Few-Shot Prompting is less effective than Zero-Shot Prompting.
   - d) Few-Shot Prompting is used for image generation, while Zero-Shot is for text.  
   **Answer:** b) Zero-Shot Prompting does not rely on specific training, while Few-Shot provides examples.

7. What is a critical element of an effective prompt?
   - a) Ambiguity
   - b) Clarity
   - c) Complexity
   - d) Length  
   **Answer:** b) Clarity

8. Which of the following is NOT a component of a prompt?
   - a) Instruction
   - b) Context
   - c) Output Indicator
   - d) Randomization  
   **Answer:** d) Randomization

9. What is the main benefit of using advanced prompting techniques?
   - a) They reduce the need for AI training.
   - b) They enhance the effectiveness of AI prompts for better outputs.
   - c) They simplify the coding process.
   - d) They eliminate the need for user input.  
   **Answer:** b) They enhance the effectiveness of AI prompts for better outputs.

10. Why is it important to maintain a consistent tone in prompts?
    - a) To confuse the model
    - b) To ensure coherent and legible responses
    - c) To make the prompt longer
    - d) To provide more examples  
    **Answer:** b) To ensure coherent and legible responses